In [1]:
# Parameters
var_list = ["default1", "default2"]
plevel_list = [False, 300]
aew_subset = "default_subset"
model_save_name = "default_modelbase1.keras"
tuner_project_name = "default_tuner_run1"






In [2]:
# Parameters
# Parameters
var_list = ["cape", "crr", "d","d", "ie", "ishf", "lsrr", "pv","pv", "q","q","r","r", "sp", "sstk", "tcw", "tcwv", "t","t", "ttr", "u","u","v","v", "vo","vo","w","w"] #ERA5 variables



plevel_list = [False, False,850,300, False, False, False, 850,300, 850,300, 850,300,False, False, False, False, 850,300,  False, 850,300,850 ,300 , 300, 850, 300,850] #pressure levels of variables

aew_subset = "12hr_before"
model_save_name = "best_model_var(300)1234.keras"
tuner_project_name = "tuner_run(300)1234"


In [3]:
#!/usr/bin/env python

# coding: utf-8


import numpy as np

import matplotlib.pyplot as plt

import xarray as xr

import sklearn

import sklearn.model_selection

import keras

from keras import layers

import keras_tuner



import tensorflow as tf

import tensorflow.keras.backend as K

keras.utils.set_random_seed(812)

2025-07-01 15:34:54.907011: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-01 15:34:55.392964: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751405695.469802  110775 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751405695.482439  110775 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751405695.581078  110775 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [4]:
import tensorflow as tf

def focal_loss(gamma=2.0, alpha=0.25):
    """Focal Loss for binary classification."""
    def loss_fn(y_true, y_pred):
        # Clip to prevent NaNs 
        y_pred = tf.clip_by_value(y_pred, tf.keras.backend.epsilon(), 1 - tf.keras.backend.epsilon())
        bce = tf.keras.backend.binary_crossentropy(y_true, y_pred)
        p_t = y_true * y_pred + (1 - y_true) * (1 - y_pred)
        modulating_factor = tf.pow(1.0 - p_t, gamma)
        alpha_factor = y_true * alpha + (1 - y_true) * (1 - alpha)
        return alpha_factor * modulating_factor * bce
    return loss_fn


In [5]:
#functions to calculate f1 score as loss function

def f1_loss_sigmoid(y_true, y_pred):
    """

    F1 metric for sigmoid output and integer encoded labels.

"""

    # compute tp, fp, and fn

    tp = K.sum(K.cast(y_true * y_pred, 'float'), axis=0)

    fp = K.sum(K.cast((1 - y_true) * y_pred, 'float'), axis=0)

    fn = K.sum(K.cast(y_true * (1 - y_pred), 'float'), axis=0)


    # precision (tp / (tp + fp))

    p = tp / (tp + fp + K.epsilon())

   # recall (tp / (tp + fn))

    r = tp / (tp + fn + K.epsilon())


# harmonic mean of precision and recall

    f1 = (2 * p * r) / (p + r + K.epsilon())

    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)

    return 1 - K.mean(f1)

In [6]:
def f1_loss_onehot(y_true, y_pred):

   """

   F1 metric for two-class output and one-hot encoded labels.

   """

   # compute tp, fp, and fn

   tp = K.sum(K.cast(y_true[:, 1] * y_pred[:, 1], 'float'), axis=0)

   fp = K.sum(K.cast((1 - y_true[:, 1]) * y_pred[:, 1], 'float'), axis=0)

   fn = K.sum(K.cast(y_true[:, 0] * (1 - y_pred[:, 0]), 'float'), axis=0)


   # precision (tp / (tp + fp))

   p = tp / (tp + fp + K.epsilon())

   # recall (tp / (tp + fn))

   r = tp / (tp + fn + K.epsilon())


   # harmonic mean of precision and recall

   f1 = (2 * p * r) / (p + r + K.epsilon())

   f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)

   return 1 - K.mean(f1)

In [7]:
import re


def add_dim(ds):
    # Extract the source file name from the dataset's encoding.
    fname = ds.encoding.get('source', '')
    # Use a regex to capture the central latitude and longitude from the filename.
    m = re.search(r'_(\-?\d+\.\d+)_(-?\d+\.\d+)\.nc$', fname)
    if m:
        lat_center = float(m.group(1))
        lon_center = float(m.group(2))
        # Assign the central coordinates and the file name as new coordinates.
        ds = ds.assign_coords(lat_center=lat_center, lon_center=lon_center, file_name=fname)
    else:
        print("File name does not match expected pattern:", fname)
    
    # Expand dims to add the 'sample' dimension and drop unnecessary variables.
    return ds.assign_coords({"sample": 1}).expand_dims(dim={"sample": 1}).drop_vars("utc_date").drop_vars("latitude").drop_vars("longitude")




    

In [8]:
import os
import xarray as xr

def open_files_zarr(list_of_vars, aew_subset="12hr_before",
                    directory="/glade/derecho/scratch/rmandava/AEW_time_location_files/",
                    plevel_list=None, zarr_store_path="zarr_data"):
    """
    Opens ERA5 NetCDF files for the given variables. For each variable (or pressure-level variant),
    it checks if a corresponding Zarr store exists in 'zarr_store_path'. If so, it loads the dataset
    from the Zarr store; if not, it opens the NetCDF files, preprocesses them, saves them to Zarr,
    and then returns the dataset.
    """
    # Create the zarr_store_path directory if it doesn't exist.
    if not os.path.exists(zarr_store_path):
        os.makedirs(zarr_store_path)
    
    datas = {}
    for num, var in enumerate(list_of_vars):
        # Determine the key and filename based on whether a pressure level is specified.
        if plevel_list:
            if plevel_list[num]:
                key = f"{var}_{int(plevel_list[num])}"
                file_pattern = f'{directory}/{var}/aew_{aew_subset}_{int(plevel_list[num])}_*.nc'
            else:
                key = var
                file_pattern = f'{directory}/{var}/aew_{aew_subset}_*.nc'
        else:
            key = var
            file_pattern = f'{directory}/{var}/aew_{aew_subset}_*.nc'
        
        # Define the zarr path for this variable.
        zarr_path = os.path.join(zarr_store_path, f"{key}.zarr")
        
        # If the Zarr dataset exists, load from it; otherwise, create it.
        if os.path.exists(zarr_path):
            print(f"Loading {key} from Zarr store.")
            ds = xr.open_zarr(zarr_path)
        else:
            print(f"Creating Zarr store for {key} from NetCDF files.")
            ds = xr.open_mfdataset(
                file_pattern,
                preprocess=add_dim,
                concat_dim="sample",
                combine="nested",
            )
            ds.to_zarr(zarr_path, mode="w")
        datas[key] = ds
    
    return datas


In [9]:
def transpose_load_concat(data_dictionary):
    # Instead of eagerly converting to NumPy arrays, keep the datasets as xarray objects.
    transposed = {}
    for key, ds in data_dictionary.items():
        var_name = key.split('_')[0].upper()
        # Do lazy transpose and add a 'features' dimension
        transposed[key] = ds[var_name].expand_dims('features').transpose('sample', 'latitude', 'longitude', 'features')
    # Concatenate along the new 'features' dimension (if multiple variables exist)
    if len(transposed) > 1:
        data = xr.concat(list(transposed.values()), dim='features',coords='minimal',compat='override')
    else:
        data = list(transposed.values())[0]
    # Use the coordinates (lat_center, lon_center) from one of the datasets.
    # They remain lazy and are not computed until needed.
    first_key = next(iter(data_dictionary))
    lat_center = data_dictionary[first_key]['lat_center']
    lon_center = data_dictionary[first_key]['lon_center']
    label = data_dictionary[first_key]['label']
    return data, label, lat_center, lon_center



In [10]:
def omit_nans(data, label, lat, lon):
    # If data is an xarray DataArray, convert it to a NumPy array
    if hasattr(data, 'values'):
        data = data.values
    maskarray = np.full(data.shape[0], True)
    # Find indices where NaNs occur
    masker = np.unique(np.argwhere(np.isnan(data))[:, 0])
    maskarray[masker] = False

    traindata = data[maskarray, ...]
    trainlabel = label[maskarray]
    lat_filtered = lat[maskarray]
    lon_filtered = lon[maskarray]
    return traindata, trainlabel, lat_filtered, lon_filtered



In [11]:
def zscore(data):

  """

  Rescaling the data using zscore (mean/std).

  Each variable gets scaled independently from others.

  Note that we will need to remove test data for formal training.

  """

  for i in range(0, data_.shape[-1]):

      data_[:, :, :, i] = (

               data_[:, :, :, i] - np.nanmean(

                     data_[:, :, :, i])) / np.nanstd(data_[:, :, :, i])

  return data_

In [12]:
def minmax(data):

   """

   Rescaling the data using min-max.

   Each variable gets scaled independently from others.

   Note that we will need to remove test data for formal training.

   """

   for i in range(0, data_.shape[-1]):

          data_[:, :, :, i] = (

              data_[:, :, :, i] - np.nanmin(data_[:, :, :, i])

          ) / (np.nanmax(data_[:, :, :, i]) - np.nanmin(data_[:, :, :, i]))
   return data

In [13]:
def random_split(data, label, split=0.3, seed=0):

   """

   Help spliting data randomly for training and testing.

   """

   np.random.seed(0)

   da_indx = np.random.permutation(data.shape[0])

   data = data[da_indx.astype(int)]

   label = label[da_indx.astype(int)]

   init_range = int(data.shape[0] * (1 - 0.3))

   return data[:init_range], label[:init_range], data[init_range:], label[init_range:]

In [14]:
def pick_loss(loss_string):
    

    lossdict = {
        "relu": tf.nn.relu,
        "tanh": tf.nn.tanh,
        "selu": tf.nn.selu,
        "sigmoid": tf.nn.sigmoid,
        "relu6": tf.nn.relu6,
        "silu": tf.nn.silu,
        "gelu": tf.nn.gelu,
        "lrelu": tf.nn.leaky_relu,
    }

    return lossdict[loss_string]

In [15]:
def compute_saliency_map(model, input_sample):
    """
    Compute a saliency map for a given input sample using a gradient-based approach.
    
    Args:
        model (tf.keras.Model): The trained Keras model.
        input_sample (numpy array): A single input sample of shape (1, height, width, channels).
    
    Returns:
        saliency (numpy array): The saliency map of shape (height, width).
    """
    # Ensure the model is in inference mode
    model.trainable = False
    input_tensor = tf.convert_to_tensor(input_sample)
    
    # Use GradientTape to record operations for automatic differentiation
    with tf.GradientTape() as tape:
        # Watch the input tensor
        tape.watch(input_tensor)
        # Get the model's prediction
        prediction = model(input_tensor)
    
    # Compute gradients of the prediction with respect to the input
    grads = tape.gradient(prediction, input_tensor)
    
    # If there are multiple channels, take the maximum absolute gradient across channels
    saliency = np.max(np.abs(grads.numpy()), axis=-1)[0]
    return saliency

In [16]:
def compute_saliency_per_channel(model, input_sample):
    """
    Computes the saliency map for each channel of a given input sample.
    
    Args:
        model (tf.keras.Model): The trained Keras model.
        input_sample (numpy array): A single input sample with shape (1, H, W, C).
        
    Returns:
        saliency_maps (numpy array): Absolute gradients with shape (H, W, C) for each channel.
        channel_importance (numpy array): Mean saliency per channel (shape: (C,)).
    """
    # Set the model to inference mode
    model.trainable = False
    input_tensor = tf.convert_to_tensor(input_sample)
    
    # Compute gradients with respect to the input sample using GradientTape
    with tf.GradientTape() as tape:
        tape.watch(input_tensor)
        prediction = model(input_tensor)
    
    # Calculate gradients: shape (1, H, W, C)
    grads = tape.gradient(prediction, input_tensor)
    
    # Remove the batch dimension: shape becomes (H, W, C)
    grads = grads.numpy()[0]
    
    # Take absolute value to measure importance (magnitude of sensitivity)
    saliency_maps = np.abs(grads)
    
    # Aggregate saliency per channel (e.g., using the mean over spatial dimensions)
    channel_importance = np.mean(saliency_maps, axis=(0, 1))
    
    return saliency_maps, channel_importance

In [17]:
number_of_features = len(var_list)

In [18]:
data = open_files_zarr(
    list_of_vars=var_list,
    aew_subset=aew_subset,
    directory="/glade/derecho/scratch/rmandava/AEW_time_location_files/",
    plevel_list=plevel_list,
    zarr_store_path="/glade/derecho/scratch/rmandava/AEW_time_location_files/Project1/zarr"
)






Loading cape from Zarr store.
Loading crr from Zarr store.
Loading d_850 from Zarr store.
Loading d_300 from Zarr store.
Loading ie from Zarr store.
Loading ishf from Zarr store.
Loading lsrr from Zarr store.
Loading pv_850 from Zarr store.
Loading pv_300 from Zarr store.
Loading q_850 from Zarr store.
Loading q_300 from Zarr store.
Loading r_850 from Zarr store.
Loading r_300 from Zarr store.
Loading sp from Zarr store.
Loading sstk from Zarr store.
Loading tcw from Zarr store.
Loading tcwv from Zarr store.
Loading t_850 from Zarr store.
Loading t_300 from Zarr store.
Loading ttr from Zarr store.
Loading u_850 from Zarr store.
Loading u_300 from Zarr store.
Loading v_850 from Zarr store.
Loading v_300 from Zarr store.
Loading vo_300 from Zarr store.
Loading vo_850 from Zarr store.
Loading w_300 from Zarr store.
Loading w_850 from Zarr store.


In [19]:
# transpose the data and concat variables

#data_, labels_ = transpose_load_concat(data)
data_, labels_, sample_lat, sample_lon = transpose_load_concat(data)

print(np.shape(data_))

(2750, 32, 32, 28)


In [20]:
# check / remove nans

data_, labels_, sample_lat, sample_lon = omit_nans(data_, labels_, sample_lat, sample_lon)






In [21]:
# Flatten spatial and channel dimensions
X_flat = data_.reshape(data_.shape[0], -1)  # (n_samples, 32*32*19)
y = labels_  


In [22]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test =  train_test_split(X_flat,y,test_size = 0.2, random_state =42)


/glade/u/home/rmandava/.local/lib/python3.10/site-packages/xarray/core/indexing.py:1642: PerformanceWarning: Slicing with an out-of-order index is generating 106 times more chunks
  return self.array[key]
/glade/u/home/rmandava/.local/lib/python3.10/site-packages/xarray/core/indexing.py:1642: PerformanceWarning: Slicing with an out-of-order index is generating 27 times more chunks
  return self.array[key]


In [23]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators =100, class_weight = 'balanced',random_state =42)
rf.fit(X_train,y_train)



RandomForestClassifier(class_weight='balanced', random_state=42)

In [24]:
from sklearn.metrics import classification_report,confusion_matrix, roc_auc_score

y_pred = rf.predict(X_test)
y_prob = rf.predict_proba(X_test)[:,1]
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_prob))

[[119   1]
 [ 13   7]]
              precision    recall  f1-score   support

         0.0       0.90      0.99      0.94       120
         1.0       0.88      0.35      0.50        20

    accuracy                           0.90       140
   macro avg       0.89      0.67      0.72       140
weighted avg       0.90      0.90      0.88       140

ROC AUC: 0.9145833333333333


In [25]:
import imblearn
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state =42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

In [26]:
from sklearn.ensemble import RandomForestClassifier
rf1 = RandomForestClassifier(n_estimators =500, class_weight = 'balanced',random_state =42)
rf1.fit(X_train_res,y_train_res)

RandomForestClassifier(class_weight='balanced', n_estimators=500,
                       random_state=42)

In [27]:
from sklearn.metrics import classification_report,confusion_matrix, roc_auc_score

y_pred = rf1.predict(X_test)
y_prob = rf1.predict_proba(X_test)[:,1]
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_prob))

[[111   9]
 [  8  12]]
              precision    recall  f1-score   support

         0.0       0.93      0.93      0.93       120
         1.0       0.57      0.60      0.59        20

    accuracy                           0.88       140
   macro avg       0.75      0.76      0.76       140
weighted avg       0.88      0.88      0.88       140

ROC AUC: 0.9139583333333334


In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'class_weight': ['balanced', 'balanced_subsample']
}
grid = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=3)
grid.fit(X_train_res, y_train_res)



In [ ]:

import pandas as pd

results_df = pd.DataFrame(grid.cv_results_)
results_df = results_df.sort_values(by='mean_test_score', ascending=False)

# View top 5 configurations
print(results_df[['mean_test_score', 'std_test_score', 'params']].head())


In [ ]:

x=5
print(x)


In [ ]:

#feature selection
from sklearn.feature_selection import SelectFromModel
selector = SelectFromModel(RandomForestClassifier(n_estimators=100)).fit(X_train_res, y_train_res)
X_train_sel = selector.transform(X_train_res)
X_test_sel = selector.transform(X_test)


In [23]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from imblearn.over_sampling import SMOTE

# 1. Split the data
X_train, X_test, y_train, y_test = train_test_split(X_flat, y, test_size=0.2, random_state=42, stratify=y)

# 2. Apply SMOTE to training data only
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# 3. Optionally apply PCA (you can turn this off)
use_pca = False
if use_pca:
    pca = PCA(n_components=100, random_state=42)
    X_train_res = pca.fit_transform(X_train_res)
    X_test_pca = pca.transform(X_test)
else:
    X_test_pca = X_test

# 4. Grid search setup
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'class_weight': ['balanced', 'balanced_subsample']
}

grid = GridSearchCV(RandomForestClassifier(random_state=42),
                    param_grid,
                    cv=3,
                    n_jobs=-1,
                    verbose=2)

# 5. Train using grid search
grid.fit(X_train_res, y_train_res)

# 6. Show top results
results_df = pd.DataFrame(grid.cv_results_)
results_df = results_df.sort_values(by='mean_test_score', ascending=False)
print("Top 5 Grid Search Results:")
print(results_df[['mean_test_score', 'std_test_score', 'params']].head())

# 7. Evaluate best model on the original test set
best_model = grid.best_estimator_

y_pred = best_model.predict(X_test_pca)
y_proba = best_model.predict_proba(X_test_pca)[:, 1]

print("\nBest Hyperparameters:\n", grid.best_params_)
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_proba))




Fitting 3 folds for each of 48 candidates, totalling 144 fits


/glade/u/home/rmandava/.local/lib/python3.10/site-packages/xarray/core/indexing.py:1642: PerformanceWarning: Slicing with an out-of-order index is generating 107 times more chunks
  return self.array[key]
/glade/u/home/rmandava/.local/lib/python3.10/site-packages/xarray/core/indexing.py:1642: PerformanceWarning: Slicing with an out-of-order index is generating 26 times more chunks
  return self.array[key]


Top 5 Grid Search Results:
    mean_test_score  std_test_score  \
41         0.943737        0.006005   
25         0.943737        0.006005   
33         0.941614        0.007506   
24         0.939490        0.009008   
40         0.939490        0.009008   

                                               params  
41  {'class_weight': 'balanced_subsample', 'max_de...  
25  {'class_weight': 'balanced_subsample', 'max_de...  
33  {'class_weight': 'balanced_subsample', 'max_de...  
24  {'class_weight': 'balanced_subsample', 'max_de...  
40  {'class_weight': 'balanced_subsample', 'max_de...  

Best Hyperparameters:
 {'class_weight': 'balanced_subsample', 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}

Confusion Matrix:
 [[112   6]
 [  7  15]]

Classification Report:
               precision    recall  f1-score   support

         0.0       0.94      0.95      0.95       118
         1.0       0.71      0.68      0.70        22

    accuracy        

In [24]:
import xgboost as xgb
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import pandas as pd
import numpy as np

# 1. Split data
X_train, X_test, y_train, y_test = train_test_split(X_flat, y, test_size=0.2, random_state=42, stratify=y)

# 2. SMOTE
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# 3. Optional PCA
use_pca = False
if use_pca:
    pca = PCA(n_components=100, random_state=42)
    X_train_res = pca.fit_transform(X_train_res)
    X_test_pca = pca.transform(X_test)
else:
    X_test_pca = X_test

# 4. Grid search over XGBoost parameters
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'scale_pos_weight': [1, (len(y_train_res) / sum(y_train_res))]  # handle imbalance
}

xgb_model = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    random_state=42,
    n_jobs=-1
)

grid = GridSearchCV(xgb_model, param_grid, cv=3, verbose=2, n_jobs=-1)
grid.fit(X_train_res, y_train_res)

# 5. Inspect best results
results_df = pd.DataFrame(grid.cv_results_).sort_values(by='mean_test_score', ascending=False)
print("Top 5 Grid Search Results:")
print(results_df[['mean_test_score', 'std_test_score', 'params']].head())

# 6. Final evaluation on test set
best_model = grid.best_estimator_
y_pred = best_model.predict(X_test_pca)
y_proba = best_model.predict_proba(X_test_pca)[:, 1]

print("\nBest Parameters:\n", grid.best_params_)
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_proba))



/glade/u/home/rmandava/.local/lib/python3.10/site-packages/xarray/core/indexing.py:1642: PerformanceWarning: Slicing with an out-of-order index is generating 107 times more chunks
  return self.array[key]
/glade/u/home/rmandava/.local/lib/python3.10/site-packages/xarray/core/indexing.py:1642: PerformanceWarning: Slicing with an out-of-order index is generating 26 times more chunks
  return self.array[key]


Fitting 3 folds for each of 36 candidates, totalling 108 fits


/glade/u/home/rmandava/.local/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [15:38:57] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/glade/u/home/rmandava/.local/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [15:38:59] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/glade/u/home/rmandava/.local/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [15:38:59] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/glade/u/home/rmandava/.local/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [15:38:59] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/glade/u/home/rmandava/.

[CV] END class_weight=balanced, max_depth=None, min_samples_leaf=2, min_samples_split=5, n_estimators=100; total time=   4.3s
[CV] END class_weight=balanced, max_depth=10, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   8.8s
[CV] END class_weight=balanced_subsample, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   9.0s
[CV] END class_weight=balanced_subsample, max_depth=10, min_samples_leaf=2, min_samples_split=5, n_estimators=200; total time=   8.5s
[CV] END learning_rate=0.01, max_depth=5, n_estimators=100, scale_pos_weight=1; total time= 4.5min
[CV] END learning_rate=0.1, max_depth=3, n_estimators=200, scale_pos_weight=2.0; total time= 2.2min
[CV] END learning_rate=0.1, max_depth=5, n_estimators=200, scale_pos_weight=1; total time= 9.9min
[CV] END learning_rate=0.1, max_depth=7, n_estimators=200, scale_pos_weight=1; total time= 3.1min
[CV] END learning_rate=0.1, max_depth=7, n_estimators=200, scale_pos_weight=2.0; tota

/glade/u/home/rmandava/.local/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [16:37:02] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[CV] END class_weight=balanced, max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   8.9s
[CV] END class_weight=balanced, max_depth=20, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   9.0s
[CV] END class_weight=balanced_subsample, max_depth=10, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   8.8s
[CV] END class_weight=balanced_subsample, max_depth=20, min_samples_leaf=2, min_samples_split=5, n_estimators=200; total time=   8.5s
[CV] END learning_rate=0.01, max_depth=7, n_estimators=100, scale_pos_weight=2.0; total time=53.5min
[CV] END class_weight=balanced, max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   8.7s
[CV] END class_weight=balanced, max_depth=20, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   4.3s
[CV] END class_weight=balanced_subsample, max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   9.

In [ ]:
import xgboost as xgb
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import pandas as pd
import numpy as np

# 1. Train-test split with stratification
X_train, X_test, y_train, y_test = train_test_split(X_flat, y, test_size=0.2, random_state=42, stratify=y)

# 2. SMOTE oversampling
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# 3. PCA (optional)
use_pca = False
if use_pca:
    pca = PCA(n_components=100, random_state=42)
    X_train_res = pca.fit_transform(X_train_res)
    X_test_pca = pca.transform(X_test)
else:
    X_test_pca = X_test

# 4. Calculate scale_pos_weight manually
neg, pos = np.bincount(y_train_res)
scale_weight = neg / pos

# 5. GridSearchCV with scale_pos_weight tuning
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.05, 0.1, 0.2],
    'scale_pos_weight': [1, scale_weight, 2*scale_weight]
}

xgb_model = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    n_jobs=-1,
    random_state=42
)

grid = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    cv=3,
    scoring='f1',
    verbose=2,
    n_jobs=-1
)

# 6. Fit the grid search
grid.fit(X_train_res, y_train_res)

# 7. View top results
results_df = pd.DataFrame(grid.cv_results_).sort_values(by='mean_test_score', ascending=False)
print("Top 5 Grid Search Results:")
print(results_df[['mean_test_score', 'std_test_score', 'params']].head())

# 8. Final evaluation on test set
best_model = grid.best_estimator_
y_pred = best_model.predict(X_test_pca)
y_proba = best_model.predict_proba(X_test_pca)[:, 1]

print("\nBest Parameters:\n", grid.best_params_)
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_proba))
